# 🚀 Amazon Nova Multimodal RAG Workshop

Welcome to this hands-on workshop where you'll learn to build a complete multimodal RAG system using Amazon Nova!

## 🎯 What You'll Learn

By the end of this workshop, you'll be able to:
- ✅ **Embed multiple images** using Nova Multimodal Embeddings
- ✅ **Perform cross-modal search** (find images with text, text with images)
- ✅ **Build tool calling systems** that integrate multimodal search
- ✅ **Use S3 Vectors** for efficient similarity search


```

## 📋 Prerequisites

- AWS Account with Bedrock access
- Basic understanding of embeddings and vector search

Let's get started! 🎉

## Step 1: Environment Setup

First, let's install the required packages and set up our AWS connections.

**What we're doing:**
- Installing Python packages for AWS SDK and image processing
- Initializing Bedrock clients for Nova models
- Setting up S3 Vectors for efficient similarity search

In [ ]:
# Install required packages
!pip install boto3 numpy pandas pillow opencv-python
!pip install -r requirementsv2.txt

In [ ]:
import boto3
import json
import base64
import os
from datetime import datetime
from typing import List, Dict, Any

# Initialize AWS clients
bedrock = boto3.client('bedrock-runtime', region_name='us-west-2')
bedrock_embedding = boto3.client('bedrock-runtime', region_name='us-east-1')
s3vectors = boto3.client('s3vectors', region_name='us-east-1')

# Configuration
NOVA_MODEL_ID = "us.amazon.nova-2-omni-v1:0"
NOVA_EMBED_MODEL_ID = "amazon.nova-2-multimodal-embeddings-v1:0"
EMBEDDING_DIM = 1024  # Using 1024 dimensions for optimal performance
VECTOR_BUCKET = f'nova-workshop-{datetime.now().strftime("%Y%m%d%H%M%S")}'
INDEX_NAME = 'multimodal-search'

print("🚀 AWS clients initialized!")
print(f"📊 Nova Model: {NOVA_MODEL_ID}")
print(f"🎨 Embedding Model: {NOVA_EMBED_MODEL_ID}")
print(f"📦 Vector Bucket: {VECTOR_BUCKET}")

## Step 2: Nova Multimodal Embeddings

Amazon Nova Multimodal Embeddings is a breakthrough model that can create unified embeddings for text, images, video, and audio content.

**Key Benefits:**
- **Unified Model**: Single model handles all content types
- **Cross-Modal Search**: Find images with text queries and vice versa
- **High Performance**: State-of-the-art accuracy with cost efficiency
- **Easy Integration**: Simple API for complex multimodal tasks

**How Embeddings Work:**
1. Convert content (text/image) into numerical vectors
2. Similar content gets similar vectors
3. Use mathematical distance to find related content

Let's create our embedding class:

In [ ]:
class NovaMultimodalEmbeddings:
    """
    Wrapper for Amazon Nova Multimodal Embeddings
    Handles text and image embeddings with error handling
    """
    
    def __init__(self):
        self.bedrock = bedrock_embedding
        self.model_id = NOVA_EMBED_MODEL_ID
        self.embedding_dim = EMBEDDING_DIM
    
    def embed_text(self, text: str) -> List[float]:
        """Generate embeddings for text content"""
        request_body = {
            "schemaVersion": "nova-multimodal-embed-v1",
            "taskType": "SINGLE_EMBEDDING",
            "singleEmbeddingParams": {
                "embeddingPurpose": "GENERIC_RETRIEVAL",
                "embeddingDimension": self.embedding_dim,
                "text": {"truncationMode": "NONE", "value": text}
            }
        }
        
        response = self.bedrock.invoke_model(
            body=json.dumps(request_body),
            modelId=self.model_id,
            contentType="application/json"
        )
        
        result = json.loads(response["body"].read())
        return result["embeddings"][0]["embedding"]
    
    def embed_image(self, image_path: str) -> List[float]:
        """Generate embeddings for image content"""
        # Read and encode image to base64
        with open(image_path, 'rb') as f:
            image_bytes = base64.b64encode(f.read()).decode('utf-8')
        
        request_body = {
            "schemaVersion": "nova-multimodal-embed-v1",
            "taskType": "SINGLE_EMBEDDING",
            "singleEmbeddingParams": {
                "embeddingPurpose": "GENERIC_RETRIEVAL",
                "embeddingDimension": self.embedding_dim,
                "image": {
                    "detailLevel": "DOCUMENT_IMAGE",
                    "format": "png",
                    "source": {"bytes": image_bytes}
                }
            }
        }
        
        response = self.bedrock.invoke_model(
            body=json.dumps(request_body),
            modelId=self.model_id,
            contentType="application/json"
        )
        
        result = json.loads(response["body"].read())
        return result["embeddings"][0]["embedding"]

# Initialize our embeddings class
embeddings = NovaMultimodalEmbeddings()
print("🎨 Nova Multimodal Embeddings ready!")

# Test with a simple example
try:
    test_embedding = embeddings.embed_text("Hello world")
    print(f"✅ Test successful! Embedding has {len(test_embedding)} dimensions")
except Exception as e:
    print(f"⚠️ Test failed: {str(e)}")

## Step 3: S3 Vectors Setup

S3 Vectors provides efficient storage and similarity search for embeddings. Think of it as a specialized database for vector operations.

**Why S3 Vectors?**
- **Scalable**: Handle millions of embeddings
- **Fast**: Optimized for similarity search
- **Cost-effective**: Pay only for what you use
- **Integrated**: Works seamlessly with AWS services

**Key Concepts:**
- **Vector Bucket**: Like a database that stores embeddings
- **Index**: Like a table with specific schema for fast search
- **Cosine Distance**: Mathematical way to measure similarity

In [ ]:
def setup_s3_vectors():
    """
    Create S3 Vectors bucket and index for storing embeddings
    
    This function:
    1. Creates a vector bucket (like creating a database)
    2. Creates an index (like creating a table with specific schema)
    3. Configures for 1024-dimensional embeddings with cosine similarity
    """
    try:
        # Create vector bucket (like creating a database)
        print("📦 Creating vector bucket...")
        s3vectors.create_vector_bucket(
            vectorBucketName=VECTOR_BUCKET,
            encryptionConfiguration={'sseType': 'AES256'}  # Encrypt data at rest
        )
        print(f"✅ Created vector bucket: {VECTOR_BUCKET}")
        
        # Create index (like creating a table with specific schema)
        print("🔍 Creating search index...")
        s3vectors.create_index(
            vectorBucketName=VECTOR_BUCKET,
            indexName=INDEX_NAME,
            dimension=EMBEDDING_DIM,  # Must match Nova's embedding dimension
            dataType='float32',       # Data type for embeddings
            distanceMetric='cosine'   # How to measure similarity
        )
        print(f"✅ Created index: {INDEX_NAME}")
        print(f"📊 Configured for {EMBEDDING_DIM}-dimensional embeddings")
        return True
        
    except Exception as e:
        if 'already exists' in str(e):
            print(f"ℹ️ Vector store already exists: {VECTOR_BUCKET}")
            return True
        print(f"❌ Error creating S3 Vectors: {str(e)}")
        return False

# Set up our vector store
print("🏗️ Setting up S3 Vectors for embedding storage...")
if setup_s3_vectors():
    print("\n🎉 S3 Vectors ready for embeddings!")
else:
    print("\n⚠️ S3 Vectors setup failed")

## Step 4: Discover and Process Content

Now let's discover images in our img folder and prepare content for embedding. We'll work with both text descriptions and actual images.

**What we're doing:**
- Finding image files in the img folder
- Creating text descriptions for different products
- Preparing metadata for better search results

**Content Strategy:**
- Mix text and images for cross-modal search
- Use descriptive text that relates to images
- Add metadata for filtering and categorization

In [ ]:
# Discover available images in img folder
IMG_FOLDER = 'img'
print(f"🔍 Discovering images in {IMG_FOLDER} folder...")

image_extensions = ('.jpg', '.jpeg', '.png', '.webp')

# Check if img folder exists
if os.path.exists(IMG_FOLDER) and os.path.isdir(IMG_FOLDER):
    image_files = [f for f in os.listdir(IMG_FOLDER) if f.lower().endswith(image_extensions)]
    # Create full paths for the images
    image_paths = [os.path.join(IMG_FOLDER, f) for f in image_files]
    
    print(f"🖼️ Found {len(image_files)} images in {IMG_FOLDER} folder:")
    for i, img in enumerate(image_files[:10], 1):  # Show first 10
        print(f"   {i}. 📸 {img}")
    
    if len(image_files) > 10:
        print(f"   ... and {len(image_files) - 10} more")
else:
    print(f"⚠️ {IMG_FOLDER} folder not found. Creating empty list.")
    image_files = []
    image_paths = []

if not image_files:
    print(f"⚠️ No images found in {IMG_FOLDER} folder. Please add some .jpg, .jpeg, or .png files.")

In [ ]:
# Create sample content with rich descriptions
# This simulates a product catalog with both text and images

print("📝 Creating sample content for multimodal search...")

content_data = [
    # Text descriptions (like product descriptions)
    {
        "type": "text",
        "content": "High-quality leather boots perfect for outdoor hiking and adventures",
        "key": "text_boots_outdoor",
        "category": "footwear",
        "description": "Product description for outdoor boots"
    },
    {
        "type": "text",
        "content": "Comfortable running sneakers with modern athletic design and cushioning",
        "key": "text_sneakers_athletic",
        "category": "footwear",
        "description": "Product description for athletic sneakers"
    },
    {
        "type": "text",
        "content": "Amazon Nova multimodal foundation model for text, image, and video processing",
        "key": "text_nova_model",
        "category": "technology",
        "description": "Information about Nova AI model"
    },
    {
        "type": "text",
        "content": "Premium footwear collection featuring various styles for different occasions",
        "key": "text_footwear_collection",
        "category": "footwear",
        "description": "General footwear collection overview"
    }
]

# Add images to our content (limit to first 10 for workshop)
print("🖼️ Adding images to content collection...")
for i, (img_file, img_path) in enumerate(zip(image_files[:10], image_paths[:10])):
    # Determine category based on filename
    category = "footwear" if any(word in img_file.lower() for word in ['boot', 'sneaker', 'shoe']) else "product"
    
    content_data.append({
        "type": "image",
        "content": img_path,  # Use full path instead of just filename
        "key": f"image_{img_file.split('.')[0]}",
        "category": category,
        "description": f"Product image: {img_file} from {IMG_FOLDER} folder"
    })

print(f"\n📊 Content collection prepared:")
print(f"   📝 Text items: {len([c for c in content_data if c['type'] == 'text'])}")
print(f"   🖼️ Image items: {len([c for c in content_data if c['type'] == 'image'])}")
print(f"   📦 Total items: {len(content_data)}")

# Show sample content
print("\n📋 Sample content:")
for item in content_data[:3]:
    print(f"   • [{item['type'].upper()}] {item['content'][:60]}...")

## Step 5: Generate and Store Embeddings

This is where the magic happens! We'll convert both text and images into embeddings using Nova, then store them in S3 Vectors.

**Key Concept:** Embeddings are numerical representations that capture the semantic meaning of content. Similar content will have similar embeddings.

**Process Flow:**
1. **For Text**: Send text to Nova → Get 1024-dimensional vector
2. **For Images**: Encode image → Send to Nova → Get 1024-dimensional vector
3. **Store**: Save vectors with metadata in S3 Vectors
4. **Index**: Enable fast similarity search

In [ ]:
# Process and store all content as embeddings
vectors = []
successful_embeddings = 0
failed_embeddings = 0

print("🔄 Processing content and generating embeddings...")
print("This may take a few moments depending on the number of items.\n")

for i, item in enumerate(content_data, 1):
    print(f"Processing {i}/{len(content_data)}: {item['type']} - {item['content'][:50]}...")
    
    try:
        # Generate embedding based on content type
        if item["type"] == "text":
            embedding = embeddings.embed_text(item["content"])
            print(f"   ✓ Generated text embedding ({len(embedding)} dimensions)")
            
        elif item["type"] == "image" and os.path.exists(item["content"]):
            embedding = embeddings.embed_image(item["content"])
            print(f"   ✓ Generated image embedding ({len(embedding)} dimensions)")
        else:
            print(f"   ⚠️ Skipping: {item['content']} (file not found)")
            failed_embeddings += 1
            continue
        
        # Create vector with rich metadata for S3 Vectors
        vector = {
            "key": item["key"],
            "data": {"float32": embedding},
            "metadata": {
                "type": item["type"],
                "content": item["content"],
                "category": item.get("category", "general"),
                "description": item.get("description", ""),
                "embedding_model": NOVA_EMBED_MODEL_ID
            }
        }
        vectors.append(vector)
        successful_embeddings += 1
        print(f"   ✓ Added to vector collection")
        
    except Exception as e:
        print(f"   ❌ Error processing {item['content']}: {str(e)}")
        failed_embeddings += 1
    
    print()  # Empty line for readability

print(f"📊 Embedding Generation Summary:")
print(f"   ✅ Successful: {successful_embeddings}")
print(f"   ❌ Failed: {failed_embeddings}")
print(f"   📦 Total vectors ready: {len(vectors)}")

In [ ]:
# Store all vectors in S3 Vectors
if vectors:
    try:
        print(f"💾 Storing {len(vectors)} embeddings in S3 Vectors...")
        
        s3vectors.put_vectors(
            vectorBucketName=VECTOR_BUCKET,
            indexName=INDEX_NAME,
            vectors=vectors
        )
        
        print(f"\n🎉 Successfully stored {len(vectors)} embeddings in S3 Vectors!")
        print(f"📊 Storage Details:")
        print(f"   📦 Bucket: {VECTOR_BUCKET}")
        print(f"   🔍 Index: {INDEX_NAME}")
        print(f"   📏 Embedding dimension: {EMBEDDING_DIM}")
        print(f"   🎯 Distance metric: cosine")
        print(f"   🔍 Ready for similarity search!")
        
    except Exception as e:
        print(f"❌ Error storing vectors: {str(e)}")
        print("Please check your AWS permissions and try again.")
else:
    print("⚠️ No vectors to store. Please check the content processing step above.")

## Step 6: Multimodal Search

Now for the exciting part - searching across different content types! You can use text to find images, or find similar text content.

**Cross-Modal Search Examples:**
- Search "leather boots" → Find boot images AND text descriptions
- Search "running" → Find sneaker images and athletic text
- Search "outdoor" → Find hiking-related content across all types

**How It Works:**
1. Convert your search query to an embedding
2. Compare with stored embeddings using cosine similarity
3. Return the most similar content (regardless of type!)
4. Filter by content type if needed

In [ ]:
def multimodal_search(query: str, content_type: str = "all", top_k: int = 5) -> List[Dict]:
    """
    Search across all content types using text query
    
    Args:
        query: Text query to search for
        content_type: Filter by 'text', 'image', or 'all'
        top_k: Number of results to return
    
    Returns:
        List of search results with metadata and similarity scores
    """
    try:
        print(f"🔍 Searching for: '{query}'")
        print(f"📊 Content filter: {content_type}")
        print(f"🎯 Max results: {top_k}")
        
        # Step 1: Convert query to embedding
        print("\n1️⃣ Converting query to embedding...")
        query_embedding = embeddings.embed_text(query)
        print(f"   ✓ Generated query embedding ({len(query_embedding)} dimensions)")
        
        # Step 2: Search S3 Vectors for similar embeddings
        print("\n2️⃣ Searching S3 Vectors for similar content...")
        response = s3vectors.query_vectors(
            vectorBucketName=VECTOR_BUCKET,
            indexName=INDEX_NAME,
            queryVector={"float32": query_embedding},
            topK=top_k * 2,  # Get extra results for filtering
            returnDistance=True,
            returnMetadata=True
        )
        
        results = response.get("vectors", [])
        print(f"   ✓ Found {len(results)} initial results")
        
        # Step 3: Filter by content type if specified
        if content_type != "all":
            print(f"\n3️⃣ Filtering results by content type: {content_type}")
            filtered_results = [r for r in results if r.get("metadata", {}).get("type") == content_type]
            print(f"   ✓ {len(filtered_results)} results after filtering")
            results = filtered_results
        
        # Step 4: Return top results
        final_results = results[:top_k]
        print(f"\n✅ Returning top {len(final_results)} results")
        
        return final_results
        
    except Exception as e:
        print(f"❌ Search error: {str(e)}")
        return []

def display_search_results(query: str, results: List[Dict], show_details: bool = True):
    """Display search results in a user-friendly format"""
    print(f"\n🔍 Search Results for: '{query}'")
    print(f"📊 Found {len(results)} results")
    print("=" * 60)
    
    if not results:
        print("No results found. Try a different search term.")
        return
    
    icons = {"text": "📝", "image": "🖼️", "video": "🎥", "audio": "🎵"}
    
    for i, result in enumerate(results, 1):
        metadata = result.get("metadata", {})
        content_type = metadata.get("type", "unknown")
        content = metadata.get("content", "N/A")
        category = metadata.get("category", "general")
        description = metadata.get("description", "")
        
        # Calculate similarity score (1 - distance = similarity)
        similarity = 1 - result.get("distance", 0)
        icon = icons.get(content_type, "❓")
        
        print(f"{i}. {icon} [{content_type.upper()}] Similarity: {similarity:.3f}")
        print(f"   Content: {content}")
        
        if show_details:
            print(f"   Category: {category}")
            if description:
                print(f"   Description: {description}")
        
        print()

print("🔍 Multimodal search functions ready!")

In [ ]:
# Let's test our multimodal search with different queries!
test_queries = [
    {"query": "leather boots", "type": "all", "description": "Find all content about leather boots"},
    {"query": "home with car", "type": "image", "description": "find homes only with car"},
    {"query": "outdoor activities", "type": "text", "description": "Find only text about outdoor activities"},
    {"query": "Nova model", "type": "all", "description": "Find content about Nova AI model"}
]

print("🧪 Testing Multimodal Search with Different Scenarios")
print("=" * 70)

for i, test in enumerate(test_queries, 1):
    print(f"\n📝 Test {i}/{len(test_queries)}: {test['description']}")
    print("-" * 50)
    
    try:
        results = multimodal_search(test["query"], test["type"], top_k=3)
        display_search_results(test["query"], results, show_details=False)
    except Exception as e:
        print(f"❌ Error in test {i}: {str(e)}")
    
    print("\n" + "=" * 70)

## Step 7: Enhanced Tool Calling with Nova

Now let's integrate our multimodal search with Nova's tool calling capabilities. This allows Nova to intelligently decide when and how to search for content.

**How Tool Calling Works:**
1. **User asks a question** → "Show me images of boots"
2. **Nova analyzes the request** → Determines it needs to search for images
3. **Nova calls our search tool** → `image_search(query="boots")`
4. **We execute the tool** → Actually perform the search
5. **Nova gets the results** → Uses search results to provide a comprehensive answer

**Benefits:**
- Nova decides which tool to use
- Automatic parameter extraction
- Intelligent response generation
- Natural conversation flow

In [ ]:
# Tool execution functions that Nova can call
def execute_multimodal_search(query: str, content_type: str = "all", max_results: int = 5) -> str:
    """
    Execute multimodal search and return formatted results for Nova
    
    This function is called by Nova when it wants to search for content.
    It returns a formatted string that Nova can understand and use.
    """
    print(f"🛠️ Executing multimodal search tool...")
    print(f"   Query: '{query}'")
    print(f"   Content type: {content_type}")
    print(f"   Max results: {max_results}")
    
    results = multimodal_search(query, content_type, max_results)
    
    if not results:
        return f"No results found for '{query}' in {content_type} content."
    
    # Format results for Nova to understand
    response_parts = [f"Found {len(results)} results for '{query}':"]
    
    for i, result in enumerate(results, 1):
        metadata = result.get("metadata", {})
        content_type_result = metadata.get("type", "unknown")
        content = metadata.get("content", "N/A")
        category = metadata.get("category", "general")
        similarity = 1 - result.get("distance", 0)
        
        response_parts.append(
            f"{i}. [{content_type_result.upper()}] {content} "
            f"(Category: {category}, Similarity: {similarity:.3f})"
        )
    
    formatted_response = "\n".join(response_parts)
    print(f"\n📊 Tool execution complete. Returning {len(results)} results to Nova.")
    
    return formatted_response

def execute_image_search(query: str, max_results: int = 3) -> str:
    """
    Execute image-specific search - specialized tool for finding images
    """
    print(f"🛠️ Executing image search tool...")
    print(f"   Query: '{query}'")
    print(f"   Max results: {max_results}")
    
    return execute_multimodal_search(query, "image", max_results)

print("🛠️ Tool execution functions ready for Nova!")

In [ ]:
# Enhanced Nova tool calling with actual tool execution
def call_nova_with_multimodal_tools(user_message: str):
    """
    Call Nova with multimodal search capabilities and execute tools when requested
    
    This function implements a complete conversation loop:
    1. Send user message to Nova
    2. Check if Nova wants to use tools
    3. Execute requested tools
    4. Send tool results back to Nova
    5. Get Nova's final response
    """
    
    print(f"🤖 Starting conversation with Nova...")
    print(f"👤 User: {user_message}")
    print("-" * 50)
    
    # Define tools that Nova can use
    tool_definitions = [
        {
            "toolSpec": {
                "name": "multimodal_search",
                "description": "Search across text and image content using semantic similarity. Use this when users ask to find, search, or look for content.",
                "inputSchema": {
                    "json": {
                        "type": "object",
                        "properties": {
                            "query": {
                                "type": "string",
                                "description": "Search query for finding relevant content"
                            },
                            "content_type": {
                                "type": "string",
                                "description": "Type of content to search: 'all' for everything, 'text' for text only, 'image' for images only",
                                "enum": ["all", "text", "image"]
                            },
                            "max_results": {
                                "type": "integer",
                                "description": "Maximum number of results to return (1-10)",
                                "minimum": 1,
                                "maximum": 10
                            }
                        },
                        "required": ["query"]
                    }
                }
            }
        },
        {
            "toolSpec": {
                "name": "image_search",
                "description": "Search specifically for images using text queries. Use this when users specifically ask for images, photos, or pictures.",
                "inputSchema": {
                    "json": {
                        "type": "object",
                        "properties": {
                            "query": {
                                "type": "string",
                                "description": "Text query to find relevant images"
                            },
                            "max_results": {
                                "type": "integer",
                                "description": "Maximum number of images to return (1-5)",
                                "minimum": 1,
                                "maximum": 5
                            }
                        },
                        "required": ["query"]
                    }
                }
            }
        }
    ]
    
    # Start conversation with Nova
    messages = [{"role": "user", "content": [{"text": user_message}]}]
    
    # Conversation loop - continue until Nova doesn't need more tools
    conversation_step = 1
    while True:
        print(f"\n🔄 Conversation Step {conversation_step}")
        
        # Send request to Nova
        request = {
            "modelId": NOVA_MODEL_ID,
            "messages": messages,
            "toolConfig": {"tools": tool_definitions},
            "additionalModelRequestFields":{'reasoningConfig': {'type': 'enabled', 'maxReasoningEffort': 'high'}}
        }
        
        response = bedrock.converse(**request)
        
        # Add Nova's response to conversation history
        messages.append(response["output"]["message"])
        
        # Check if Nova wants to use tools
        tool_requests = []
        for content in response["output"]["message"]["content"]:
            if "toolUse" in content:
                tool_requests.append(content["toolUse"])
            elif 'reasoningContent' in content:
                print("Reasoning Content :")
                print(content['reasoningContent']['reasoningText']['text'])
        
        if not tool_requests:
            # No tools requested - Nova has the final answer
            print("\n🤖 Nova's Response:")
            for content in response["output"]["message"]["content"]:
                if "text" in content:
                    print(content["text"])
            break
        
        # Execute requested tools
        print(f"\n🛠️ Nova requested {len(tool_requests)} tool(s)")
        tool_results = []
        
        for tool_request in tool_requests:
            tool_name = tool_request["name"]
            tool_input = tool_request["input"]
            tool_id = tool_request["toolUseId"]
            
            print(f"\n🔧 Executing tool: {tool_name}")
            print(f"📝 Parameters: {tool_input}")
            
            # Execute the appropriate tool
            if tool_name == "multimodal_search":
                result = execute_multimodal_search(
                    tool_input["query"],
                    tool_input.get("content_type", "all"),
                    tool_input.get("max_results", 5)
                )
            elif tool_name == "image_search":
                result = execute_image_search(
                    tool_input["query"],
                    tool_input.get("max_results", 3)
                )
            else:
                result = f"Unknown tool: {tool_name}"
            
            print(f"\n📊 Tool Result Preview:")
            print(f"{result[:200]}{'...' if len(result) > 200 else ''}")
            
            # Format tool result for Nova
            tool_results.append({
                "toolResult": {
                    "toolUseId": tool_id,
                    "content": [{"text": result}]
                }
            })
        
        # Send tool results back to Nova
        messages.append({"role": "user", "content": tool_results})
        conversation_step += 1
        
        # Safety check to prevent infinite loops
        if conversation_step > 5:
            print("\n⚠️ Conversation limit reached. Ending conversation.")
            break

print("🤖 Enhanced Nova tool calling system ready!")

## Step 8: Test Your Multimodal AI Assistant

Time to see your creation in action! Try asking Nova questions that require searching through your content.

**Great Test Queries:**
- "Show me images of boots" → Tests image search
- "Find content about running shoes" → Tests multimodal search
- "What outdoor footwear do you have?" → Tests semantic understanding
- "Tell me about Nova's capabilities" → Tests text search

**What to Watch For:**
- Nova deciding which tool to use
- Tool execution with parameters
- Search results being returned
- Nova's intelligent response using the results

In [ ]:
# Test queries for the workshop
workshop_queries = [
    "Show me houses with car",
    "What outdoor footwear do you have?",
    "Tell me about Nova's capabilities",
]

print("🧪 Testing Your Multimodal AI Assistant")
print("=" * 70)
print("\nWatch how Nova intelligently decides which tools to use!")
print("Each test demonstrates different aspects of multimodal search.")

for i, query in enumerate(workshop_queries, 1):
    print(f"\n\n📝 Test {i}/{len(workshop_queries)}")
    print(f"🎯 Query: '{query}'")
    print("=" * 70)
    
    try:
        call_nova_with_multimodal_tools(query)
    except Exception as e:
        print(f"❌ Error in test {i}: {str(e)}")
        print("This might be due to API limits or configuration issues.")
    
    print("\n" + "=" * 70)
    
    # Add a small pause between tests for readability
    import time
    time.sleep(1)

print("\n🎉 Testing complete! Your multimodal AI assistant is working!")

## 🎯 Workshop Summary

Congratulations! You've successfully built a complete multimodal RAG system with Amazon Nova. Here's what you accomplished:

### ✅ What You Built:

1. **🎨 Multimodal Embeddings**: Used Nova to create unified embeddings for text and images
2. **📦 Vector Storage**: Set up S3 Vectors for efficient similarity search
3. **🔍 Cross-Modal Search**: Enabled searching images with text and vice versa
4. **🛠️ Tool Integration**: Connected Nova with your search capabilities
5. **🤖 AI Assistant**: Created an intelligent system that can understand and search multimodal content

### 🚀 Key Concepts Learned:

- **📊 Embeddings**: Numerical representations that capture semantic meaning
- **🔄 Cross-Modal Search**: Finding related content across different media types
- **📏 Vector Similarity**: Using mathematical distance to find similar content
- **🛠️ Tool Calling**: Enabling AI models to use external functions
- **🎭 Multimodal AI**: Systems that understand multiple types of content


Great job completing this workshop! 🎉

## 🧹 Cleanup (Optional)

When you're done experimenting, you can clean up the AWS resources to avoid charges.

**What gets cleaned up:**
- S3 Vectors bucket and index
- All stored embeddings
- Search infrastructure

**Note**: This is permanent! You'll need to re-run the workshop to recreate everything.

In [ ]:
def cleanup_workshop_resources():
    """
    Clean up AWS resources created during the workshop
    """
    print("🧹 Starting workshop cleanup...")
    print("⚠️ This will permanently delete all workshop resources!")
    
    try:
        # Delete the vector index first
        print("\n1️⃣ Deleting vector index...")
        s3vectors.delete_index(
            vectorBucketName=VECTOR_BUCKET,
            indexName=INDEX_NAME
        )
        print(f"   ✅ Deleted index: {INDEX_NAME}")
        
        # Delete the vector bucket
        print("\n2️⃣ Deleting vector bucket...")
        s3vectors.delete_vector_bucket(vectorBucketName=VECTOR_BUCKET)
        print(f"   ✅ Deleted bucket: {VECTOR_BUCKET}")
        
        print("\n🎉 Cleanup completed successfully!")
        print("\n📋 Resources that were cleaned up:")
        print(f"   📦 Vector bucket: {VECTOR_BUCKET}")
        print(f"   🔍 Vector index: {INDEX_NAME}")
        print(f"   🎨 All stored embeddings ({successful_embeddings} items)")
        print(f"   💾 Search infrastructure")
        
        print("\n💡 To use the system again, re-run the entire workshop notebook.")
        
    except Exception as e:
        print(f"❌ Error during cleanup: {str(e)}")
        print("\n🔧 Manual cleanup may be required:")
        print(f"   - Delete S3 Vectors bucket: {VECTOR_BUCKET}")
        print(f"   - Delete vector index: {INDEX_NAME}")
        print("   - Check AWS Console for any remaining resources")

# Uncomment the line below to run cleanup
cleanup_workshop_resources()

print("💡 To clean up resources, uncomment and run: cleanup_workshop_resources()")
print("⚠️ Warning: This will permanently delete all workshop data!")